# Parallelization - Workflow Interface

In order to test the scaling of a particular system on a cluster, we have to compare the optimal performance of the cluster for an increasing number of resources (nodes) used.

A sensible measure for gauging the performance of different resources/parallelization settings is the average electronic time step. This is especially useful compared to running a full calculation when we are testing the scaling for a very demanding calculation, as this might not be able to finish in the walltime for our baseline resource specification (ideally a single core). In light of this, we will simply perform a static calculation with a very demanding electronic convergence condition (1e-8). This will allow for a sufficient amount of electronic steps to average over.

The cell below launches an interface for submitting workflows to a test mongoDB server. Be sure to change the default INCAR settings if desired, or change the settings of the two `Custodian` `ErrorHandler`'s we use for these workflows.

**Note: Because the Binder server seems to have issues connecting to the mongoDB database, submitting workflows is not possible when using this notebook from Binder.**

In [1]:
from vscworkflows.handlers.core import ParallelizationTestMonitor, JobTerminator

try:
    CLUSTER_DICT = {
        "leibniz": {
            "launchpad": LaunchPad.from_dict({
                'host': 'mongodb+srv://testcluster-au9d3.gcp.mongodb.net',
                'port': 27017,
                'name': 'test_leibniz',
                'username': 'testuser',
                'password': 'testpassword',
                'ssl': True,
                'authsource': 'admin'
            }),
            "scratch_dir": "/scratch/antwerpen/202/vsc20248/",
            "cores_per_node": 28
        },
        "hopper": {
            "launchpad": LaunchPad.from_dict({
                'host': 'mongodb+srv://testcluster-au9d3.gcp.mongodb.net',
                'port': 27017,
                'name': 'test_hopper',
                'username': 'testuser',
                'password': 'testpassword',
                'ssl': True,
                'authsource': 'admin'
            }),
            "scratch_dir": "/scratch/antwerpen/202/vsc20248/",
            "cores_per_node": 20
        }
    }
except ConfigurationError:
    print("Failed to connect to mongoDB server specified by the Launchpad. " +
          "Please check your connection.")

# Defaults for the parallelization settings
user_incar_settings = {"ALGO": "Normal", "ENCUT":350, "EDIFF": 1e-8}

handlers = [ParallelizationTestMonitor(max_elec_step_time=500, max_elec_steps=10),
            JobTerminator(errors_subset_to_catch=["subspacematrix", "zpotrf"], timeout=100)]

# Launch the interface
%run -i online_workflow_interface.py

Output()

Output()